## Assigment: Neural network basics

Soft deadline: 16.09.18 at 23.59

Hard deadline: 18.09.18 at 23.59

For this task I intentionally provide no boilerplate code, because very puprpose of this task is getting you comforatable with basic code template for desiging NNs in pytorch. I higly recommend you to revisit all the last seminar materials.

#### Task

* Implement simple **fully-convolutional** neural architecture for classification. Make sure it is small enought to run on your home machine.
* Provide dataset visulization.
* Provide train/test split and validation

#### Requirements

* Architecture should derive from `torch.nn.Module`
* Use `torch.utils.data.Dataset` and `torch.utils.data.DataLoader`. But if you manage co simplify this step using dataset `torchivision`, I will only encourage you.
* Implement at least one data transformer, but make sure it is useful for classification task.
* Use FashionMNIST dataset https://github.com/zalandoresearch/fashion-mnist
* Make sure you can fix random seed for all components of your code to make experiments reproducible
* Since you architecure should be fully-convolutional, make sure it does not depend on input size.

In [17]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import random
from IPython import display
from sklearn import datasets, preprocessing

In [41]:
BATCH_SIZE = 50

transform = transforms.ToTensor()
        #TODO: transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = torchvision.datasets.FashionMNIST("./", train=True, transform=transform, download=True)
test_set = torchvision.datasets.FashionMNIST("./", train=False, transform=transform, download=True)
print(train_set.__len__)
print(test_set.__len__)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE)
fst_batch = next(iter(train_loader))

# print(fst_batch[0].size(), fst_batch[1].size())
# print(fst_batch[0][0])
    
    

<bound method MNIST.__len__ of Dataset FashionMNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./
    Transforms (if any): ToTensor()
    Target Transforms (if any): None>
<bound method MNIST.__len__ of Dataset FashionMNIST
    Number of datapoints: 10000
    Split: test
    Root Location: ./
    Transforms (if any): ToTensor()
    Target Transforms (if any): None>
torch.Size([50, 1, 28, 28]) torch.Size([50])
